## Notes
- Input: `outputs/clean1_loginDetails.csv`
- Output: `outputs/clean2_loginDetails.csv`

In [1]:
import pandas as pd
import re
import numpy as np

In [2]:
df = pd.read_csv("outputs/clean1_loginDetails.csv")

In [3]:
df.columns

Index(['adm_no', 'sex', 'dept', 'borrower_type', 'entry_date', 'entry_time'], dtype='object')

In [4]:
# Functions to get department name and year from dept field
def getYear(s):
    _ = re.search(r'[0-9]{4}', s)
    if(_ is not None):
        return _[0]
    return np.nan

def getDept(s):
    _ = re.sub(r"[0-9]{4}", "", s)
    _ = _.replace("-", " ").strip()
    if (_.find("nan") != -1):
        return np.nan
    return _

In [5]:
# Create dictionary to replace dept field with department name and year 

# pd.set_option('display.max_rows', None) # to show all rows
unq_dept = df["dept"].drop_duplicates()
replace_dept = {}
for x in unq_dept:
    replace_dept[x] = {"year": getYear(str(x)), "branch": getDept(str(x))}

# print(replace_dept)

In [7]:
# Create two cols department and batch from dept column

depts = []
years = []

for i in df["dept"]:
    try:
        depts.append(replace_dept[i]["branch"])
        years.append(replace_dept[i]["year"])
    except KeyError:
        depts.append(None)
        years.append("")

df["department"] = depts
df["batch"] = years

# verification
# df.columns

In [9]:
# drop old dept column
df2 = df.drop('dept', axis=1)

# verification
# df2.columns

In [11]:
# replacing CSE with CS for Computer Science department
df2["department"] = df2["department"].replace(['CSE'], 'CS')

In [13]:
# finding unique department names
_depts = []
for i in df2["department"].drop_duplicates():
    if i is np.nan:
        continue
    _depts.append(str(i))
print(_depts)

['ME', 'EC', 'IT', 'CS', 'EEE', 'ELECTRONICS AND COMMUNICATION', 'M.Tech  CSCL', 'ECONOMICS', 'MTech CSE CL', 'COMPUTER SCIENCE', 'GENERAL', 'M Tech  CSCL', 'Ph.D', 'ELECTRICAL', 'INFORMATION TECHNOLOGY', 'MECHANICAL', 'CENTRAL LIBRARY', 'CIVIL', 'MATHEMATICS', 'M.Tech ME ROB', 'CE', 'MTech ME ROB']


In [17]:
# from the unique names select only required
_depts = ['ME', 'EC', 'IT', 'CS', 'EEE','EE', 'CE']

In [15]:
null_data = df[df["dept"].isnull()]

In [16]:
null_data.to_csv("outputs/clean2_loginDetails_missingDept.csv", index=False)